In [6]:
import numpy as np
import os, sys
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import datetime as DT
import matplotlib.dates as mdates

#sys.path.remove(sys.path[0])
#sys.path.append('/Users/ajc/Core/codes/simulator/debug/ats/tools/utils/')

import plot_water_table2D

import importlib as imp
print (plot_water_table2D.__file__)
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 12}
matplotlib.rc('font', **font)
import parse_ats
print (parse_ats.__file__)

/Users/ajc/Core/codes/simulator/repos/amanzi/src/physics/ats/tools/utils/plot_water_table2D.py
/Users/ajc/Core/codes/simulator/repos/amanzi/src/physics/ats/tools/utils/parse_ats.py


In [7]:
import re
def sorted_nicely( l ):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

In [8]:
path = "/Users/ajc/Core/simulations/hillslope/rain_advection/SIM_04/"


In [9]:
#Dec 22, 2020: ProjectionsO
Simulations = [['S3_P20_%sX'%(i+1) for i in range(1)], ['S3_P34_%sX'%(i+1) for i in range(1)],
               ['S3_P50_%sX'%(i+1) for i in range(1)],
               ['S3_P20_%sX'%(i+3) for i in range(1)], ['S3_P34_%sX'%(i+3) for i in range(1)],
               ['S3_P50_%sX'%(i+3) for i in range(1)]]

Simulations = [['S6_P20_%sX'%(i+1) for i in range(1)], ['S6_P34_%sX'%(i+1) for i in range(1)],
               ['S6_P50_%sX'%(i+1) for i in range(1)],
               ['S6_P20_%sX'%(i+3) for i in range(1)], ['S6_P34_%sX'%(i+3) for i in range(1)],
               ['S6_P50_%sX'%(i+3) for i in range(1)]]

Simulations = [['S3_P20_%sX'%(i+1) for i in range(1)], ['S3_P20_%sX'%(i+3) for i in range(1)]]

print (Simulations)
Sim_name = ['S3_P20_1x','S3_P34_1x','S3_P50_1x','S3_P20_3x','S3_P34_3x','S3_P50_3x']
Sim_name = ['S3_P20_1x','S3_P20_3x']

[['S3_P20_1X'], ['S3_P20_3X']]


In [10]:
#We note that topography influences the spatial distribution of soil water content, 
#and the availability of soil water influences the type of vegetation that can grow.

In [11]:
def getData(i,x_pos,Tref):
    time_, wl_, td_, z_ = dict(), dict(), dict(), dict()
    Time, WL,TD,z_elev = [], [],[],[]
    #location = Location[0]
    key = Sim_name[i] + '_' + str(x_pos)
    
    for s in Simulations[i]:
        
        infile = os.path.join(path,s)
        t, wl, td, z = plot_water_table2D.water_table2D(infile,x_pos=x_pos,Tref=Tref)
        a1 = len(t)%365
        a2 = len(t) - a1
        t = t[:a2]
        wl = wl[:a2]
        td = td[:a2]
        Time.append(t*365.25/365.)
        WL.append(wl)
        TD.append(td)
        z_elev.append(z)
        
    if (x_pos == -1):
        key = Sim_name[i] + '_' + 'center'
    else:
        key = Sim_name[i] + '_' + str(x_pos) +'m'
    time_[key+'_time'] = np.concatenate(Time)
    wl_[key+'_wl'] = np.concatenate(WL)
    td_[key+'_td'] = np.concatenate(TD)
    z_[key+'_z'] = np.array(z_elev)
    
    print ('Length2:',i, x_pos,key)

    return time_,wl_, td_,z_

In [12]:
Time = dict()
WaterLevel = dict()
ThawDepth = dict()
Elevation = dict()

from multiprocessing import Pool
from functools import partial
num_processors = 2

pool=Pool(processes = num_processors)

def call_data(x_pos,Tref):
    global DataS1, DataS2#, DataS3, DataS4, DataS5, DataS6
    prod_x=partial(getData, x_pos=x_pos,Tref=Tref)
    #DataS1, DataS2, DataS3,DataS4, DataS5, DataS6 = pool.map(prod_x,[i for i in range(len(Simulations))])
    #Data = [DataS1, DataS2,DataS3,DataS4, DataS5, DataS6]
    
    DataS1, DataS2  = pool.map(prod_x,[i for i in range(len(Simulations))])
    Data = [DataS1, DataS2]
    return Data

Cell okay:  0 30
Cell okay:  0 30
Length2: 0 0 S3_P20_1x_0m
Length2: 1 0 S3_P20_3x_0m


In [13]:

def getData_All(x_pos,Tref):
    DataSimOrig = dict()
    Data = call_data(x_pos,Tref)
    for i in range(len(Sim_name)): #number of simulations
        for k in range(4): #number of keys = [time, water level, thaw depth, elevation]
            
            key = list(Data[i][k].keys())[0]
            z_k = list(Data[i][3].keys())[0]
            
            z = Data[i][3][z_k]
            
            if 'time' in key:
                DataSimOrig[key] = np.array([t for t in Data[i][k][key]])
                DataSimOrig[key+'_yr'] = np.reshape(DataSimOrig[key],(-1,73)).mean(axis=1)
                #print ('Time: ',DataSimOrig[key].shape)
            if 'wl' in key:
                DataSimOrig[key] = np.array([t + z[0] for t in Data[i][k][key]])
            if 'td' in key:
                DataSimOrig[key] = np.array([t + z[0] for t in Data[i][k][key]])
                print ('Z',z)
                DataSimOrig[key+'_max']  = np.reshape(DataSimOrig[key],(-1,73)).min(axis=1)
            if 'z' in key:
                DataSimOrig[z_k] = z
    return DataSimOrig

In [14]:
DataSim_x100m = getData_All(x_pos=0,Tref=273.55)

Z [[5.]]
Z [[5.]]


In [15]:
#DataSim_x500m = getData_All(x_pos=49)


In [16]:
DataSim_x100m.keys()

dict_keys(['S3_P20_1x_0m_time', 'S3_P20_1x_0m_time_yr', 'S3_P20_1x_0m_wl', 'S3_P20_1x_0m_td', 'S3_P20_1x_0m_td_max', 'S3_P20_1x_0m_z', 'S3_P20_3x_0m_time', 'S3_P20_3x_0m_time_yr', 'S3_P20_3x_0m_wl', 'S3_P20_3x_0m_td', 'S3_P20_3x_0m_td_max', 'S3_P20_3x_0m_z'])

In [17]:
#outfile='/Users/ajc/Documents/MyPapers/TroughBC/figures/Nov20/'
%matplotlib qt
def plot_data():
    fig, axs = plt.subplots(1,3, figsize=(8,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)

    plt.tight_layout()
    axs = axs.ravel()
    #plt.axhline(y=Z_c,linestyle='--',color='g')
    DataSim = [DataSim_x100m]

    year_on, thaw = True, False
    Colors = ['k','r','k','r','r','r']
    marks = ['solid','dashed']
    X = np.linspace(0,365,365)
    for k in range(1):
        dat = DataSim[k]
        
        for i in range(2):
            if i < 3:
                c = i
            else:
                c = i - 3
            Keys = np.reshape(list(dat.keys()),(-1,6))
            t_k, wl_k, td_k, z_k = Keys[i][0],Keys[i][2], Keys[i][3], Keys[i][5]
            z = dat[z_k]
            #print (td_k,dat[td_k],z)
            #break
            if year_on and not thaw:
                #print (j, i, t_k, wl_k, z_k)
                #axs[2*j+k].plot(dat[t_k][s:e],dat[wl_k][s:e],label=Sim_name[i],color=Colors[i])
                #print (dat[wl_k][s:e])
                if i < 3:
                    #axs[0].plot(dat[t_k],dat[wl_k]-z,label=Sim_name[i],color=Colors[i],linestyle=marks[0])
                    axs[0].plot(dat[t_k],dat[td_k]-z,label=Sim_name[i],color=Colors[i],linestyle=marks[0])
                else:
                    axs[0].plot(dat[t_k],dat[wl_k]-z,label=Sim_name[i],color=Colors[i],linestyle=marks[1])
                    #axs[0].plot(dat[t_k],dat[td_k]-z,label=Sim_name[i],color=Colors[i],linestyle=marks[1])

            #axs[2*j+k].axhline(y=z[0],linestyle='--',color='g')
            #axs[2*j+k].set_ylim(4.4,5.0)
            #axs[c].set_ylim(-1, 0)
            #axs[2*j+k].set_xticklabels(np.arange(0,366,73))
            #if (k==0):
            #    axs[c].set_ylabel('Water level [m]',fontsize=12, fontweight='normal')
    #axs[5].legend(labelspacing=.05)
    #axs[4].set_xlabel('Day of year',fontsize=12, fontweight='normal')
    #axs[5].set_xlabel('Day of year',fontsize=12, fontweight='normal')

    #plt.savefig(outfile+'watertable.pdf', bbox_inches='tight', dpi=500)
#plt.show()

#plot 2nd row for high snow years

In [18]:
plot_data()

findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.


In [ ]:
601/365.

In [ ]:
#%matplotlib qt
outfile='/Users/ajc/Documents/MyPapers/TroughHydrodynamics/figures/Dec22_uniform_om/'

def get_moving_average_forward(dat,key, m=5):
        d_avg = []
        len1 = len(dat[key])
        for i in range(len1):
            if i < len1 - m:
                s1 = np.mean(dat[key][i:i+m])
            else:
                s1 = np.mean(dat[key][i:i+m])
            d_avg.append(s1)
        return d_avg
    
def get_moving_average(dat, key, m=5):
    d_avg = []
    len1 = len(dat[key])
    for i in range(len1):
        if i < m:
            s1 = np.mean(dat[key][:i+1])
        else:
            s1= np.mean(dat[key][i-m:i])

        d_avg.append(s1)
    return d_avg
Sim_nameL = ['NF (t)', 'FF (t)', 'HF (t)','NF (c)', 'FF (c)', 'HF (c)']
Sim_nameL = ['zero-head (t)','specified-head (t)','dynamic-head (t)', 'zero-head (c)','specified-head (c)','dynamic-head (c)'
            ,'zero-head (r)','specified-head (r)','dynamic-head (r)']
#Sim_nameL = ['zero head (t)', 'free-flow (t)', 'hybrid-flow (t)','no-flow (c)', 'free-flow (c)', 'hybrid-flow (c)']
def plot_ALT_MicroLocations():
    fig, axs = plt.subplots(1,1, figsize=(5,3.5), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    plt.tight_layout()
    #axs = axs.ravel()
    #plt.axhline(y=Z_c,linestyle='--',color='g')
    DataSim = [DataSimTrough,DataSimCenter,DataSimRim]
    
    Colors = ['b','k','r','r']
    X = np.linspace(0,365,73)
    dry_year = 22
    wet_year =91
    avg_year = [27,44,76]
    Y = [wet_year,dry_year,avg_year[1]]
    #avg_year = [27,44,76]
    #Y = avg_year
    Lstyle = ['solid', 'dashed', ':']
    for i in range(0,3):
        dat = DataSim[i]
        #print (dat)
        Keys = np.reshape(list(dat.keys()),(-1,6))
        for j in range(3):
            t_k, td_k, z_k = Keys[j][0], Keys[j][3], Keys[j][5]
            #print (t_k, td_k)
            t_k = t_k + '_yr'
            td_k = td_k + '_max'
            dd = get_moving_average(dat,key=td_k,m=5)
            Z_c = dat[z_k][0,0]
            
            xx = (dd[0] - dd[-1])/dd[0]*100
            ddd = [k - dd[0] for k in dd]
            #print (xx, dd[0]-dd[-1])
            if (i ==0 ):
                xx = [round(x1,2) for x1 in Z_c-dd]
                #print (Z_c,dd[-1])
                print (",".join(map(str,xx)))#
                ,dat[td_k],Z_c-dat[td_k][-1], len(dat[td_k]),dat[td_k][64])
                print ("-----")
            axs.plot(2006 + dat[t_k],dd - Z_c,color=Colors[j],label=Sim_nameL[3*i+j],linestyle=Lstyle[i])
            axs.set_ylim(-2.0,0)
            axs.set_yticks(np.arange(0,-2.5,-0.5))
            xlabels = np.arange(0,2.5,0.5)
            axs.set_yticklabels(xlabels)
            
            axs.set_xlabel('Calendar year',fontsize=12, fontweight='normal')
            axs.set_ylabel('Maximum thaw depth [m]',fontsize=12, fontweight='normal')
            axs.set_xlim((2006.5, 2100.5))
            #axs[j].set_xlim((Y[j], Y[j]+1))
            #axs[j].set_xticklabels(np.arange(0,366,73))
        plt.legend(loc='lower left', fontsize=8)
        plt.savefig(outfile+'thawdepth-micro_locations.pdf', bbox_inches='tight', dpi=500)
        #plt.show()

        #plot 2nd row for high snow years

In [ ]:
plot_ALT_MicroLocations() #used to produced recent plots

In [ ]:
%matplotlib qt
outfile='/Users/ajc/Documents/MyPapers/TroughBC/figures/'

Sim_nameL = ['NF (t)', 'FF (t)', 'HF (t)','NF (c)', 'FF (c)', 'HF (c)']
Sim_nameL = ['no-flow (t)', 'free-flow (t)', 'hybrid-flow (t)','no-flow (c)', 'free-flow (c)', 'hybrid-flow (c)']
def plot_ALT_Percentage():
    fig, axs = plt.subplots(1,1, figsize=(5,3), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    plt.tight_layout()
    #axs = axs.ravel()
    #plt.axhline(y=Z_c,linestyle='--',color='g')
    DataSim = [DataSimTrough,DataSimCenter]
  
    Colors = ['k','b','r','r']
    X = np.linspace(0,365,73)
    dry_year = 22
    wet_year =91
    avg_year = [27,44,76]
    Y = [wet_year,dry_year,avg_year[1]]
    #avg_year = [27,44,76]
    #Y = avg_year
    Lstyle = ['solid', 'dashed']
    for i in range(2):
        dat = DataSim[i]
        #print (i)
        Keys = np.reshape(list(dat.keys()),(-1,6))
        for j in range(3):
            t_k, td_k, z_k = Keys[j][0], Keys[j][3], Keys[j][5]
            #print (t_k, td_k)
            t_k = t_k + '_yr'
            td_k = td_k + '_max'
            dd = get_moving_average(dat,key=td_k)
            Z_c = dat[z_k][0,0]
            #print (Z_c)
            print (dd[0])
            ddd = [Z_c - kk for kk in dd]
            xx = [(kk - ddd[0])/ddd[0]*100 for kk in ddd]
            ddd = [k - dd[0] for k in dd]
            #print (xx, dd[0]-dd[-1])
            #if (i ==0 ):
            #    print (dd - Z_c)
            axs.plot(2006 + dat[t_k],xx,color=Colors[j],label=Sim_nameL[3*i+j],linestyle=Lstyle[i])
            #axs.set_ylim(-2.75,0)
            #plt.gca().invert_yaxis()
            #axs.set_yticks(np.arange(0,-2.75,-0.5))
            #xlabels = np.arange(0,2.75,0.5)
            #print (xlabels)
            #axs.set_yticklabels(xlabels)
            #plt.ylim(3.2,3.5)
            axs.set_xlabel('Calendar year',fontsize=12, fontweight='normal')
            axs.set_ylabel('Maximum thaw depth [m]',fontsize=12, fontweight='normal')
            axs.set_xlim((2006, 2100))
            #axs[j].set_xlim((Y[j], Y[j]+1))
            #axs[j].set_xticklabels(np.arange(0,366,73))
        plt.legend()
        #plt.savefig(outfile+'thawdepth-trough-dashed-center.pdf', bbox_inches='tight', dpi=500)
        #plt.show()

        #plot 2nd row for high snow years

In [ ]:
plot_ALT_Percentage()

In [ ]:
outfile='/Users/ajc/Documents/MyPapers/TroughBC/figures/'

fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()
#axs = axs.ravel()
#plt.axhline(y=Z_c,linestyle='--',color='g')
Z_c = DataSimOrig[z_k][0,0]
year_on, thaw = True, True
Colors = ['k','b','r','r']
X = np.linspace(0,365,73)
dry_year = 22
wet_year =91
avg_year = [27,44,76]
Y = [wet_year,dry_year,avg_year[1]]
avg_year = [27,44,76]
Y = avg_year
def get_moving_average(key, m=5):
    d_avg = []
    len1 = len(DataSimOrig[key])
    for i in range(len1):
        if i < len1 - 5:
            s1 = np.mean(DataSimOrig[td_k][i:i+5])
        else:
            s1 = np.mean(DataSimOrig[td_k][i:i+5])
        d_avg.append(s1)
    return d_avg
for j in range(3):
    t_k = list(Data[i][0].keys())[0] + '_yr'
    td_k = list(Data[j][2].keys())[0] + '_max'
    dd = get_moving_average(td_k)
    #plt.plot(DataSimOrig[t_k],DataSimOrig[td_k],color=Colors[j],label=Sim_name[j])
    ###plt.plot(2006 + DataSimOrig[t_k],dd,color=Colors[j],label=Sim_name[j])
    #plt.axhline(y=Z_c,linestyle='--',color='g')
    #print (Z_c)
    xx = (dd[0] - dd[-1])/dd[0]*100
    ddd = [k - dd[0] for k in dd]
    print (xx, dd[0]-dd[-1])
    plt.plot(2006 + DataSimOrig[t_k],dd - Z_c,color=Colors[j],label=Sim_name[j])
    axs.set_ylim(-2.25,0)
    #plt.gca().invert_yaxis()
    axs.set_yticks(np.arange(0,-2.25,-0.5))
    xlabels = np.arange(0,2.25,0.5)
    #print (xlabels)
    axs.set_yticklabels(xlabels)
    #plt.ylim(3.2,3.5)
    plt.xlabel('Calendar year',fontsize=12, fontweight='normal')
    plt.ylabel('Maximum thaw depth [m]',fontsize=12, fontweight='normal')
    #plt.xlim((2006, 2100))
    #axs[j].set_xlim((Y[j], Y[j]+1))
    #axs[j].set_xticklabels(np.arange(0,366,73))
plt.legend()
#plt.savefig(outfile+'thawdepth-trough.pdf', bbox_inches='tight', dpi=800)
plt.show()

#plot 2nd row for high snow years